# NYBG DATA

In [1]:
! pip install tensorflow
! pip install numpy
! pip install matplotlib
! pip install pandas
! pip install scikit-learn
! pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import necessary libraries
#pip3 install tensorflow
import tensorflow
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

2024-02-18 13:30:07.524818: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-18 13:30:07.830284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-18 13:30:07.830343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-18 13:30:07.885041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-18 13:30:08.003765: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-18 13:30:08.006000: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
filename_train = "./bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)
filename_test = "./bttai-nybg-2024/BTTAIxNYBG-test.csv"
df_test = pd.read_csv(filename_test, header = 0)
filename_val = "./bttai-nybg-2024/BTTAIxNYBG-validation.csv"
df_val = pd.read_csv(filename_val, header = 0)

## Provided Example Starting Notebook

In [4]:
classes = list(df_train["classLabel"].unique())
classes

['occluded-specimens',
 'microscope-slides',
 'illustrations-color',
 'animal-specimens',
 'live-plants',
 'biocultural-specimens',
 'illustrations-gray',
 'mixed-pressed-specimens',
 'ordinary-pressed-specimens',
 'micrographs-transmission-light']

In [5]:
image_directory = './bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'  # 


# Preprocessing function to load and process images
def load_and_preprocess_image(filename, target_size=(224, 224)):
    img_path = os.path.join(image_directory, filename)
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Model expects a batch of images
    return img_array / 255.0  # Normalize to [0, 1]

# Load and preprocess images
df_train['imageFile'] = df_train['imageFile'].apply(load_and_preprocess_image)

: 

In [ ]:


# Apply preprocessing to all images
df['imageData'] = df['imageFile'].apply(load_and_preprocess_image)

# Split dataset into training and validation sets
### Note: This is a common step in ML training, but in this challenge, since the validation set is provided separately, there is no need to call this function to distinguish between validation and train set.
# train_df, validate_df = train_test_split(df, test_size=0.2, random_state=42)

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Note: No augmentation for validation data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, batch_size=32):
    datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=image_directory,
        x_col='imageFile',
        y_col='classLabel',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training and validation
train_dataset = df_to_dataset(train_df, train_datagen)
validation_dataset = df_to_dataset(validate_df, validation_datagen)

# This setup is now ready for training with model.fit using the train_dataset and validation_dataset
